In [1]:
import tensorflow as tf
import os
import glob

2024-03-23 15:31:12.365109: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-23 15:31:12.368322: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-23 15:31:12.409567: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-23 15:31:13.350434: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
train_images_path_pattern = os.path.join("train", "*", "*.jpg")
train_image_file_paths = glob.glob(train_images_path_pattern)

test_images_path_pattern = os.path.join("test", "*", "*.jpg")
test_image_file_paths = glob.glob(test_images_path_pattern)

train_image_file_paths[:5], test_image_file_paths[:5]

(['train/muffin/img_4_7.jpg',
  'train/muffin/img_2_490.jpg',
  'train/muffin/img_3_100.jpg',
  'train/muffin/img_3_791.jpg',
  'train/muffin/img_3_408.jpg'],
 ['test/muffin/img_3_1240.jpg',
  'test/muffin/img_3_50.jpg',
  'test/muffin/img_0_31.jpg',
  'test/muffin/img_2_305.jpg',
  'test/muffin/img_3_1013.jpg'])